# LeRobot Dataset Reader (Quick Inspector)

**Purpose:** Quickly load a LeRobot dataset (from the Hugging Face Hub or a local folder), peek at its structure, print shapes/dtypes, read meta info/stats, and visualize a few samples.

- Works with LeRobot `v2.1` and `v3` APIs (tries both import paths).
- Supports **Hub** or **local** loading (use the `root` argument for local folders).
- Includes optional streaming and quick plots for actions and images.

> References: LeRobot v3 loading examples and API tips are documented here: https://huggingface.co/docs/lerobot/en/lerobot-dataset-v3


## 0) Prerequisites

Run this once in your environment/Colab. If you already have these packages, you can skip.

```bash
pip install -U lerobot datasets pandas matplotlib pillow pyarrow
# Optional (only if you also need RLDS or TF-based utilities):
# pip install -U tensorflow tensorflow_datasets
# If you plan to train:
# pip install -U torch torchvision torchaudio
```


In [5]:
# 1) Config — set how to find your dataset

# # Option A — From the Hugging Face Hub (recommended)
# USE_HUB = True
REPO_ID = "xX-Conan-Xx/pick_up_transparent_bottle"  # e.g., "your-username/your-dataset"

# Option B — Local dataset directory (the folder that contains `meta/`, `data/`, `videos/`)
# If you recorded/converted locally or downloaded an archive, set LOCAL_ROOT to that folder.
# If left empty, we'll fall back to the default cache (~/.cache/huggingface/lerobot/REPO_ID).
LOCAL_ROOT = "/media/zeyu/082b281d-ee9b-bc4b-be11-a1acf8642a75/Data/huggingface/lerobot/xX-Conan-Xx/pick_up_transparent_bottle/"  # e.g., "/path/to/my/le_rbt_dataset"

# Set to True to stream from Hub without downloading locally (v3 only)
STREAM = False

# How many frames to sample for the quick plots
MAX_FRAMES = 200


In [3]:
# 2) Imports and compatibility (v3 first, then v2 path as fallback)
from pathlib import Path
import os, json, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Do NOT set any specific matplotlib styles or colors here (keep defaults).

# Try v3 import path first
LeRobotDataset = None
StreamingLeRobotDataset = None
try:
    from lerobot.datasets.lerobot_dataset import LeRobotDataset as _LRD
    LeRobotDataset = _LRD
    from lerobot.datasets.streaming_dataset import StreamingLeRobotDataset as _SLRD
    StreamingLeRobotDataset = _SLRD
    API_VERSION = "v3"
except Exception:
    # Fallback to older (v2.x) import path
    try:
        from lerobot.common.datasets.lerobot_dataset import LeRobotDataset as _LRD
        LeRobotDataset = _LRD
        API_VERSION = "v2.x"
    except Exception as e:
        raise ImportError(
            "Could not import LeRobotDataset from either v3 or v2 paths. "
            "Please ensure `pip install -U lerobot` succeeded."
        )

API_VERSION


'v2.x'

In [6]:
# 3) Load the dataset (Hub/local/streaming)

def _resolve_repo_id_from_local(local_root: str) -> str:
    """Try to read repo_id from meta/info.json if REPO_ID isn't set."""
    try:
        info_p = Path(local_root) / "meta" / "info.json"
        if info_p.exists():
            with open(info_p, "r") as f:
                info = json.load(f)
            return info.get("repo_id", "")
    except Exception:
        pass
    return ""

if not REPO_ID and LOCAL_ROOT:
    REPO_ID = _resolve_repo_id_from_local(LOCAL_ROOT)

if STREAM:
    if StreamingLeRobotDataset is None:
        raise RuntimeError("StreamingLeRobotDataset is only available in LeRobot v3.")
    if not REPO_ID:
        raise ValueError("For streaming, set REPO_ID (e.g., 'user/dataset').")
    dataset = StreamingLeRobotDataset(REPO_ID)
else:
    # Normal (non-streaming) mode; pass root if provided to prefer a local folder
    kwargs = {}
    if LOCAL_ROOT:
        kwargs["root"] = LOCAL_ROOT
    if not REPO_ID:
        raise ValueError("Please set REPO_ID (and optionally LOCAL_ROOT).")
    dataset = LeRobotDataset(REPO_ID, **kwargs)

print(f"Loaded dataset with API={API_VERSION}")
print("Repo ID:", REPO_ID)
try:
    length = len(dataset)
    print("Number of indexable frames:", length)
except Exception as e:
    print("len(dataset) not available:", e)


Resolving data files:   0%|          | 0/45 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loaded dataset with API=v2.x
Repo ID: xX-Conan-Xx/pick_up_transparent_bottle
Number of indexable frames: 5680


In [18]:
# 4) Peek at one random sample and list keys/shapes

idx = 1  # change if you like
sample = dataset[idx]
print("Available keys in sample:")
for k in sorted(sample.keys()):
    v = sample[k]
    shape = getattr(v, "shape", None)
    dtype = getattr(v, "dtype", None)
    print(f"  - {k:35s} | shape={tuple(shape) if shape is not None else '<no shape>'} | dtype={dtype}")

# Common keys: 'observation.state', 'action', 'timestamp', and possibly multi-camera images
state = sample.get('state', None)
action = sample.get('actions', None)
if state is not None and action is not None:
    print("\nState dim:", int(state.shape[-1]))
    print("Action dim:", int(action.shape[-1]))


Available keys in sample:
  - actions                             | shape=(7,) | dtype=torch.float32
  - episode_index                       | shape=() | dtype=torch.int64
  - frame_index                         | shape=() | dtype=torch.int64
  - image                               | shape=(3, 256, 256) | dtype=torch.float32
  - index                               | shape=() | dtype=torch.int64
  - state                               | shape=(7,) | dtype=torch.float32
  - task                                | shape=<no shape> | dtype=None
  - task_index                          | shape=() | dtype=torch.int64
  - timestamp                           | shape=() | dtype=torch.float32
  - wrist_image                         | shape=(3, 256, 256) | dtype=torch.float32

State dim: 7
Action dim: 7


In [30]:
dataset

LeRobotDataset({
    Repository ID: 'xX-Conan-Xx/pick_up_transparent_bottle',
    Number of selected episodes: '45',
    Number of selected samples: '5680',
    Features: '['image', 'wrist_image', 'state', 'actions', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',

In [ ]:
# 5) Read meta info/stats directly from disk (best-effort)
# We'll try several likely locations to find `meta/info.json` and `meta/stats.json`.

def guess_dataset_dir(repo_id: str, local_root: str | None) -> Path | None:
    candidates = []
    if local_root:
        candidates.append(Path(local_root))
    # Default HF cache location for LeRobot
    if repo_id:
        candidates.append(Path.home() / ".cache" / "huggingface" / "lerobot" / repo_id)
        # Some environments use nested folder `~/.cache/huggingface/lerobot/lerobot/<name>`
        candidates.append(Path.home() / ".cache" / "huggingface" / "lerobot" / "lerobot" / repo_id.split('/')[-1])
    for p in candidates:
        if (p / "meta" / "info.json").exists():
            return p
    return None

root_dir = guess_dataset_dir(REPO_ID, LOCAL_ROOT if LOCAL_ROOT else None)
if root_dir is None:
    print("Could not locate meta files on disk. If you are streaming, meta may not be local.")
else:
    print("Dataset directory:", root_dir)
    info_p = root_dir / "meta" / "info.json"
    stats_p = root_dir / "meta" / "stats.json"
    if info_p.exists():
        info = json.loads(Path(info_p).read_text())
        print("\nmeta/info.json:\n", json.dumps(info, indent=2)[:2000], "..." if info_p.stat().st_size > 2000 else "")
    if stats_p.exists():
        stats = json.loads(Path(stats_p).read_text())
        print("\nmeta/stats.json (truncated):\n", json.dumps(stats, indent=2)[:2000], "..." if stats_p.stat().st_size > 2000 else "")


In [ ]:
# 6) Quick look at actions over the first N frames (contiguous chunk)
# This is a coarse check for scale/magnitude and to verify action dimensionality.

N = min(MAX_FRAMES, len(dataset) if hasattr(dataset, '__len__') else 128)
if N <= 0:
    print("Dataset appears empty or non-indexable.")
else:
    A = []
    for i in range(N):
        s = dataset[i]
        a = s.get('action', None)
        if a is None:
            break
        try:
            a_np = a.detach().cpu().numpy()
        except Exception:
            a_np = np.array(a)
        A.append(a_np)
    if len(A) > 0:
        A = np.stack(A)  # [N, action_dim]
        plt.figure()
        plt.plot(A)
        plt.title("Actions over first N frames")
        plt.xlabel("t (frame index)")
        plt.ylabel("action dims (stacked)")
        plt.show()
        print("Action array shape:", A.shape)
    else:
        print("No 'action' key found in samples, or empty dataset.")


In [ ]:
# 7) Visualize one image if available (commonly 'observation.images.front_left')
import re

def find_first_image_key(keys):
    # Heuristic: any key that contains 'images' and returns a 3D tensor [C,H,W] or [H,W,C]
    for k in keys:
        if 'images' in k or 'image' in k:
            return k
    return None

img_key = find_first_image_key(sample.keys())
if img_key is None:
    print("No image-like key detected in the sample.")
else:
    img = sample[img_key]
    try:
        arr = img.detach().cpu().numpy()
    except Exception:
        arr = np.array(img)

    # If channel-first, switch to HWC for matplotlib
    if arr.ndim == 3 and arr.shape[0] in (1, 3):
        arr = np.transpose(arr, (1, 2, 0))
    plt.figure()
    if arr.ndim == 2:  # grayscale
        plt.imshow(arr, cmap='gray')
    else:
        plt.imshow(arr)
    plt.title(f"Sample image — key: {img_key}")
    plt.axis('off')
    plt.show()


In [ ]:
# 8) (Optional) Temporal windows via delta_timestamps (v3 feature)
# Recreate dataset with a small temporal stack for one camera if present.

try:
    from lerobot.datasets.lerobot_dataset import LeRobotDataset as _V3Check  # will fail on v2
    # Try to pick a likely image key
    key = None
    for k in sample.keys():
        if 'observation.images' in k:
            key = k
            break
    if key:
        delta_timestamps = {key: [-0.2, -0.1, 0.0]}
        kwargs = {}
        if LOCAL_ROOT:
            kwargs['root'] = LOCAL_ROOT
        ds_stack = LeRobotDataset(REPO_ID, delta_timestamps=delta_timestamps, **kwargs)
        s2 = ds_stack[0]
        if key in s2:
            print(f"Temporal stack for {key}:", s2[key].shape, "[T, C, H, W]")
    else:
        print("No obvious image key to demo delta_timestamps.")
except Exception as e:
    print("delta_timestamps demo skipped (requires v3 API):", e)


In [ ]:
# 9) Helper: quick consistency checks

def check_state_action_dims(ds, n_checks: int = 3):
    ok = True
    for i in range(min(n_checks, len(ds) if hasattr(ds, '__len__') else n_checks)):
        s = ds[i]
        st = s.get('observation.state', None)
        ac = s.get('action', None)
        if st is None or ac is None:
            print(f"[idx {i}] missing state/action keys -> state={st is not None}, action={ac is not None}")
            ok = False
            continue
        sdim = int(st.shape[-1])
        adim = int(ac.shape[-1])
        print(f"[idx {i}] state_dim={sdim}, action_dim={adim}")
        if sdim < adim:
            print("  WARNING: state dim < action dim (unusual unless using compact actions).")
            ok = False
    return ok

_ = check_state_action_dims(dataset, n_checks=3)


---

### Notes

- If you converted **LIBERO (RLDS)** to LeRobot and see **state dim = 8** but **action dim = 7**, this is expected if your conversion/export only maps the first 7 action channels (e.g., 7-DoF arm without gripper or with a merged gripper command). Always verify the conversion script’s action slicing.  
- For more on loading and streaming in v3, see the official docs ("Load a dataset for training" and "Stream a dataset") — they show the exact keys you’ll get, like `observation.state`, `action`, and camera tensors.  
- If your dataset is fully local, set `USE_HUB=False` and **only** `LOCAL_ROOT`. If you still see issues, open `meta/info.json` and check the `repo_id` — pass that in `REPO_ID` as well.

Happy debugging! :)
